This notebook is designed for preprocessing our image data to fit the random forest or other tree model. I do not know how to directly put the image data into a tree model. But I also know the result of doing that won't be great. Convnet may be a good feature finder for a tree model. Since it scan the image patch by patch, so it may get some feature in that patch, not a single pixel, which may contain too much noise.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [7]:
#import training data
X_train_rotate = np.load("X_train_rotate.npy")


In [15]:
target_train_rotate = np.load("target_train_rotate.npy")

In [ ]:
train = pd.read_json("train.json")
X_angle = train['inc_angle']
np.save('X_angle',X_angle)

In [2]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, BatchNormalization
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Using TensorFlow backend.
/Users/yaqiongzhang/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
#define our model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(BatchNormalization())
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.4))

    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.4))
    
    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

#     #Dense Layers
#     gmodel.add(Dense(512))
#     gmodel.add(Activation('relu'))
#     gmodel.add(Dropout(0.2))

#     #Dense Layer 2
#     gmodel.add(Dense(256))
#     gmodel.add(Activation('relu'))
#     gmodel.add(Dropout(0.2))

#     #Sigmoid Layer
#     gmodel.add(Dense(1))
#     gmodel.add(Activation('sigmoid'))

#     mypotim=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#     gmodel.compile(loss='binary_crossentropy',
#                   optimizer=mypotim,
#                   metrics=['accuracy'])
    gmodel.summary()
    return gmodel


# def get_callbacks(filepath, patience=2):
#     es = EarlyStopping('val_loss', patience=patience, mode="min")
#     msave = ModelCheckpoint(filepath, save_best_only=True)
#     return [es, msave]

# file_path = "model_weights.hdf5"
# callbacks = get_callbacks(filepath=file_path, patience=5)

In [6]:
import os
gmodel=getModel()
gmodel.load_weights('model_weights.hdf5',by_name=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
batch_normalization_9 (Batch (None, 73, 73, 64)        256       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 71, 71, 64)        36928     
_________________________________________________________________
batch_normalization_10 (Batc (None, 71, 71, 64)        256       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 33, 33, 64)        36928     
__________

In [8]:
rf_gen = gmodel.predict(X_train_rotate)

In [9]:
rf_gen.shape

(4812, 14400)

In [12]:
rf_gen.dtype

dtype('float32')

In [38]:
from sklearn.ensemble import RandomForestClassifier
for n_estimators in [100, 500, 1000, 2000, 5000]:
    rf = RandomForestClassifier(n_estimators=500, oob_score=True)

    rf = rf.fit(rf_gen, target_train_rotate)
    print('Our OOB prediction of accuracy is: {oob}%'.format(oob=rf.oob_score_ * 100))

Our OOB prediction of accuracy is: 79.4472152950956%
Our OOB prediction of accuracy is: 79.98753117206982%
Our OOB prediction of accuracy is: 79.96674979218619%
Our OOB prediction of accuracy is: 80.42394014962593%
Our OOB prediction of accuracy is: 80.17456359102243%


In [31]:
X_test =np.load('X_test.npy')
test_id=np.load('test_id.npy')

In [26]:
rf_gen_test = gmodel.predict(X_test)

In [29]:
predict_test=rf.predict(rf_gen_test)

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    1.5s finished


In [37]:
submission = pd.DataFrame()
submission['id']=test_id
submission['is_iceberg']=predict_test.reshape((predict_test.shape[0]))
submission.to_csv('sub_rf.csv', index=False)

In [28]:
from xgboost import XGBClassifier
xgboost = XGBClassifier()

ModuleNotFoundError: No module named 'xgboost'